In [ ]:
import os
import json
import numpy as np
from tensorflow.keras.models import load_model, Model
from tensorflow.keras.layers import Dense, Input
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications.imagenet_utils import preprocess_input

def auto_configure_model(model_path, dataset_path):
    model = load_model(model_path)
    temp_datagen = ImageDataGenerator(rescale=1. / 255)
    temp_generator = temp_datagen.flow_from_directory(dataset_path, shuffle=False)
    num_classes = temp_generator.num_classes

    input_shape = model.input_shape[1:]
    is_color = input_shape[-1] == 3

    if num_classes == 2 and model.output_shape[-1] == 1:
        input_layer = Input(shape=input_shape)
        x = input_layer
        for layer in model.layers[:-1]:
            x = layer(x)
        output = Dense(2, activation='softmax')(x)
        model = Model(inputs=input_layer, outputs=output)
        model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

    is_regression = model.output_shape[-1] == 1 and 'sigmoid' not in [
        layer.activation.__name__ for layer in model.layers if hasattr(layer, 'activation')
    ]

    return {
        'model': model,
        'target_size': input_shape[:2],
        'color_mode': 'rgb' if is_color else 'grayscale',
        'class_mode': 'binary' if model.output_shape[-1] == 1 and not is_regression else 'categorical',
        'preprocessing_function': preprocess_input if any(layer.name.startswith(('vgg', 'resnet', 'inception')) for layer in model.layers) else None,
        'is_regression': is_regression,
        'num_classes': num_classes
    }

def evaluate_model(config, dataset_path):
    test_gen = ImageDataGenerator(
        rescale=1./255,
        preprocessing_function=config['preprocessing_function']
    ).flow_from_directory(
        dataset_path,
        target_size=config['target_size'],
        color_mode=config['color_mode'],
        batch_size=32,
        class_mode=config['class_mode'],
        shuffle=False
    )

    results = config['model'].evaluate(test_gen)
    return results[0] if config['is_regression'] else results[1]

def save_results(metric, config, output_file):
    result_data = {
        "accuracy": round(metric * 100, 2) if not config['is_regression'] else None,
        "mse": round(metric, 4) if config['is_regression'] else None,
        "task": "regression" if config['is_regression'] else "classification",
        "num_classes": config['num_classes']
    }

    with open(output_file, 'w') as f:
        json.dump(result_data, f, indent=4)

if __name__ == "__main__":
    try:
        with open("uploads/config.json", "r") as f:
            config_data = json.load(f)

        model_path = config_data["models"][0]
        dataset_path = config_data["datasets"][0]
        output_path = os.path.join("uploads", "accuracy1.json")

        config = auto_configure_model(model_path, dataset_path)
        metric = evaluate_model(config, dataset_path)
        save_results(metric, config, output_path)

        model_path = config_data["models"][1]
        dataset_path = config_data["datasets"][1]
        output_path = os.path.join("uploads", "accuracy2.json")

        config = auto_configure_model(model_path, dataset_path)
        metric = evaluate_model(config, dataset_path)
        save_results(metric, config, output_path)

    except Exception as e:
        with open("uploads/error.log", "w") as f:
            f.write(str(e))


Found 243 images belonging to 3 classes.
Found 243 images belonging to 3 classes.


c:\Users\mrroy\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


8/8 ━━━━━━━━━━━━━━━━━━━━ 1s 22ms/step - AUC: 0.9892 - Precision: 0.9064 - Recall: 0.9032 - accuracy: 0.9032 - loss: 0.2104
Found 243 images belonging to 3 classes.
Found 243 images belonging to 3 classes.
8/8 ━━━━━━━━━━━━━━━━━━━━ 1s 23ms/step - AUC: 0.9892 - Precision: 0.9064 - Recall: 0.9032 - accuracy: 0.9032 - loss: 0.2104
